<a href="https://colab.research.google.com/github/ceciliacasarella/bertopic_modeling/blob/main/BertopicOptimizationExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##  Open the notebook in Colab to be sure you properly visualize plotly interactive graph

In [2]:
!pip install git+https://github.com/ceciliacasarella/bertopic_modeling.git --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 34.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 98.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [3]:
!git clone https://github.com/ceciliacasarella/bertopic_modeling/

Cloning into 'bertopic_modeling'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (136/136), done.
remote: Total 289 (delta 50), reused 251 (delta 30), pack-reused 0
Receiving objects: 100% (289/289), 1.79 MiB | 16.06 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [4]:
import sys
sys.path.append('/bertopic_modeling')

In [5]:
import bertopic_modeling as opt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
!pip install gdown
!gdown 1YGcv-a026sMdkX_q-whqhyh1ybiH6eCI
! openssl enc -aes-256-cbc -pbkdf2 -iter 100000 -d -in datasets.tar.gz.enc | tar xz
! sudo apt-get install jq
! for i in 1 2 3 4 ; do jq length datasets/json/$i.json ; done
! for i in 1 2 3 4 ; do cat datasets/json/$i.json | jq '.[0]' ; done
! cat datasets/sq/* #print the open question for each dataset

Downloading...
From: https://drive.google.com/uc?id=1YGcv-a026sMdkX_q-whqhyh1ybiH6eCI
To: /content/datasets.tar.gz.enc
100% 215k/215k [00:00<00:00, 72.3MB/s]
enter AES-256-CBC decryption password:
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.provenance'
tar: Ignoring unknown extende

## How to access MLflow UI in Google Colab

### Option 1: more complex, creates tunnel for public url

In [14]:
!pip install pyngrok --quiet

from pyngrok import ngrok
from getpass import getpass

get_ipython().system_raw("mlflow ui --port 5000 &")

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
NGROK_AUTH_TOKEN = getpass('Enter the ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# Open an HTTPs tunnel on port 5000 for http://localhost:5000
ngrok_tunnel = ngrok.connect(addr="5000", proto="http", bind_tls=True)
print("MLflow Tracking UI:", ngrok_tunnel.public_url)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.7/718.7 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Enter the ngrok authtoken: ··········


MLflow Tracking UI: https://61ff-35-201-184-46.ngrok.io


### Option 2: more convenient option, the downside is that we won't have a GUI.

In [7]:
get_ipython().system_raw("mlflow ui --port 5000 &")

## Optimize BERTopic using Python module

### Example 1

In [13]:
import pandas as pd

run_name = "bert-topic-first-dataset-1"
label_lower = 6
label_upper = 15
max_evals = 100
penalty= 0.3
experiment_name = "First-Dataset-Model-BERT-1"
data_path = "/content/datasets/json/1.json"
tracking_uri = "http://127.0.0.1:5000"

topic_model, sentence_topic_df = opt.clustering_eval_mlflow(
                                                tracking_client = tracking_uri,
                                                run_name = run_name,
                                                sentences = None,
                                                data_path = data_path,
                                                label_lower = label_lower,
                                                label_upper = label_upper,
                                                penalty = penalty,
                                                max_evals = max_evals,
                                                experiment_name = experiment_name)


Experiment name set on MLFlow Tracking Server: 
 First-Dataset-Model-BERT-1
100%|██████████| 100/100 [08:59<00:00,  5.40s/trial, best loss: 0.10976837865055387]
<Run: data=<RunData: metrics={'dbvc_score': 0.4414743962504552,
 'label_count': 7.0,
 'loss': 0.10976837865055387}, params={'min_cluster_size': '16',
 'n_components': '34',
 'n_neighbors': '77',
 'random_state': '42'}, tags={'mlflow.runName': 'bert-topic-first-dataset-1',
 'mlflow.source.name': '/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'root'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/363597088781125639/1dce173492904683884202ae98367fc1/artifacts', end_time=1699820121081, experiment_id='363597088781125639', lifecycle_stage='active', run_id='1dce173492904683884202ae98367fc1', run_name='bert-topic-first-dataset-1', run_uuid='1dce173492904683884202ae98367fc1', start_time=1699820121037, status='FINISHED', user_id='root'>, inputs=<RunInputs: dataset_inpu

In [16]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,109,-1_interessante originale sembra molto_molto i...,"[interessante originale sembra molto, molto in...","[molto interessante e originale, sembra molto ..."
1,0,360,0_possibilità vincita giochi_giochi possibilit...,"[possibilità vincita giochi, giochi possibilit...",[che ci sono 5 giochi e quindi c’è più possibi...
2,1,279,1_colori molto belli_colori molto_piacciono co...,"[colori molto belli, colori molto, piacciono c...",[i colori del biglietto sono molto accattivant...
3,2,113,2_tanti giochi solo biglietto_stesso biglietto...,"[tanti giochi solo biglietto, stesso biglietto...","[ci sono più giochi nello stesso biglietto, ci..."
4,3,75,3_giochi solo gratta vinci_giochi unico gratta...,"[giochi solo gratta vinci, giochi unico gratta...",[mi piace il fatto che ci siano 5 giochi in un...
5,4,31,4_djjdjdjdj djdjjdjd_loba como yo no_como yo n...,"[djjdjdjdj djdjjdjd, loba como yo no, como yo ...",[non lo so forse nulla è rimasto lo stesso di ...
6,5,26,5_piace molto semplicità gioco_molto gioco sem...,"[piace molto semplicità gioco, molto gioco sem...",[il gioco sembra assai interessante perché ren...


In [17]:
topic_model.visualize_barchart()

In [19]:
data_path = "/content/datasets/json/1.json"
sentences = opt.build_data(data_path)
topic_model.visualize_documents(sentences)

# If you do not see the plot properly please click on "Reset Axes" icon at the top right corner of the plot

In [20]:
topic_model.visualize_topics()

In [15]:
topic_model.visualize_heatmap()

In [14]:
sentence_topic_df

,text,topic
0,i colori vivaci e allegri che invitano a grattare,-1
1,le regole del gioco sono divertenti e mi piacc...,5
2,che ci sono molte possibilità di giocare,0
3,"ci sono diversi giochi, ti da l’impressione di...",0
4,"ci sono molti giochi, davvero molti, non ho al...",0
...,...,...
988,è semplice e coinvolgente sperando che si vinca,5
989,non ne vado matta perché preferisco altri giochi,0
990,colotehgf gg fcdbdbc f nnd fdjx nxbf xndb fj...,4
991,tutti i simboli colorati e allegri che danno p...,1


### Example 2

In [22]:
run_name = "bert-topic-second-dataset-2"
label_lower = 8
label_upper = 15
max_evals = 100
penalty= 0.3
experiment_name = "Second-Dataset-Model-BERT"
data_path = "/content/datasets/json/2.json"
tracking_uri = "http://127.0.0.1:5000"

topic_model, sentence_topic_df = opt.clustering_eval_mlflow(
                                                tracking_client = tracking_uri,
                                                run_name = run_name,
                                                sentences = None,
                                                data_path = data_path,
                                                label_lower = label_lower,
                                                label_upper = label_upper,
                                                penalty = penalty,
                                                max_evals = max_evals,
                                                experiment_name = experiment_name)


Experiment name set on MLFlow Tracking Server: 
 Second-Dataset-Model-BERT
100%|██████████| 100/100 [08:53<00:00,  5.34s/trial, best loss: 0.08585858585858586]
<Run: data=<RunData: metrics={'dbvc_score': 0.6076692549996168,
 'label_count': 11.0,
 'loss': 0.08585858585858586}, params={'min_cluster_size': '12',
 'n_components': '9',
 'n_neighbors': '32',
 'random_state': '42'}, tags={'mlflow.runName': 'bert-topic-second-dataset-2',
 'mlflow.source.name': '/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'root'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/721912917928445791/25a92c5130824a498a3f33787645cd21/artifacts', end_time=1699822286191, experiment_id='721912917928445791', lifecycle_stage='active', run_id='25a92c5130824a498a3f33787645cd21', run_name='bert-topic-second-dataset-2', run_uuid='25a92c5130824a498a3f33787645cd21', start_time=1699822286155, status='FINISHED', user_id='root'>, inputs=<RunInputs: dataset_inp

In [23]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,85,-1_10 euro troppo_euro troppo_10 euro_po troppo,"[10 euro troppo, euro troppo, 10 euro, po trop...","[sembrano essere giochi già visti, nulla di nu..."
1,0,230,0_nulla piace niente_nulla piace nulla_piace n...,"[nulla piace niente, nulla piace nulla, piace ...",[mi piace tutto non c è niente che non mi piac...
2,1,192,1_troppo confusionario troppe_confusionario tr...,"[troppo confusionario troppe, confusionario tr...","[un po' troppo confusionario, troppe modalità ..."
3,2,99,2_niente piace biglietto nulla_nulla piace big...,"[niente piace biglietto nulla, nulla piace big...",[non c'è nulla che non mi piace di questo bigl...
4,3,91,3_niente piace gratta vinci_nulla piace gratta...,"[niente piace gratta vinci, nulla piace gratta...",[niente mi piace tutto in questo gratta e vinc...
5,4,80,4_tante possibilità vincita_vincita_vincere po...,"[tante possibilità vincita, vincita, vincere p...","[troppo confuso poche possibilità di vincita, ..."
6,5,71,5_prezzo troppo caro_10 euro po_costo dieci eu...,"[prezzo troppo caro, 10 euro po, costo dieci e...",[il prezzo di euro 10.00 lo trovo un po' tropp...
7,6,66,6_troppo colorato colore_piaccionl colori trop...,"[troppo colorato colore, piaccionl colori trop...","[se non è vincente,che avevo speso soldi per n..."
8,7,35,7_grafica sembra troppo affollata_piace molto ...,"[grafica sembra troppo affollata, piace molto ...",[forse la grafica del biglietto è un po confus...
9,8,23,8_cambierei nulla va bene_nulla cambierei nien...,"[cambierei nulla va bene, nulla cambierei nien...","[niente non cambierei nulla, va bene così, mi ..."


In [24]:
topic_model.visualize_barchart()

In [26]:
data_path = "/content/datasets/json/2.json"
sentences = opt.build_data(data_path)
topic_model.visualize_documents(sentences)
# If you do not see the plot properly please click on "Reset Axes" icon at the top right corner of the plot

In [27]:
topic_model.visualize_topics()

In [28]:
topic_model.visualize_heatmap()

In [29]:
sentence_topic_df

,text,topic
0,troppe immagini insieme creano un pochino di c...,1
1,le regole del gioco sono buone e non c’è posto...,1
2,i colori della cartella sono troppo infantili,6
3,"ti fa credere di aver più possibilità, per io ...",4
4,"un po' confusionario, troppi colori, non ho al...",1
...,...,...
985,costa troppo sperando che offra più soldi,5
986,preferisco altri gratta e vinci questo non mi ...,3
987,novb h b v bskdn x dx dn ff zb zidb djzd z x...,-1
988,nulla in particolare lo dico sinceramente,0


### Example 3

In [7]:
get_ipython().system_raw("mlflow ui --port 5000 &")
run_name = "bert-topic-third-dataset-2"
label_lower = 15
label_upper = 25
max_evals = 100
penalty= 0.3
experiment_name = "Third-Dataset-Model-BERT"
data_path = "/content/datasets/json/3.json"
tracking_uri = "http://127.0.0.1:5000"

topic_model, sentence_topic_df = opt.clustering_eval_mlflow(
                                                tracking_client = tracking_uri,
                                                run_name = run_name,
                                                sentences = None,
                                                data_path = data_path,
                                                label_lower = label_lower,
                                                label_upper = label_upper,
                                                penalty = penalty,
                                                max_evals = max_evals,
                                                experiment_name = experiment_name)


2023/11/13 22:16:04 INFO mlflow.tracking.fluent: Experiment with name 'Third-Dataset-Model-BERT' does not exist. Creating a new experiment.


Experiment name set on MLFlow Tracking Server: 
 Third-Dataset-Model-BERT
100%|██████████| 100/100 [29:29<00:00, 17.70s/trial, best loss: 0.2345758354755784]
<Run: data=<RunData: metrics={'dbvc_score': 0.47689857705947675,
 'label_count': 16.0,
 'loss': 0.2345758354755784}, params={'min_cluster_size': '25',
 'n_components': '26',
 'n_neighbors': '83',
 'random_state': '42'}, tags={'mlflow.runName': 'bert-topic-third-dataset-2',
 'mlflow.source.name': '/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'root'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/112386840390305329/338e1fc4faad4a88a3e515c182831c10/artifacts', end_time=1699914240832, experiment_id='112386840390305329', lifecycle_stage='active', run_id='338e1fc4faad4a88a3e515c182831c10', run_name='bert-topic-third-dataset-2', run_uuid='338e1fc4faad4a88a3e515c182831c10', start_time=1699914240792, status='FINISHED', user_id='root'>, inputs=<RunInputs: dataset_inputs

Instructions for updating:
Use tf.identity with explicit device placement instead.


In [8]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,365,-1_spazi verdi_molti spazi_poca pulizia_puliti,"[spazi verdi, molti spazi, poca pulizia, pulit...",[vivo in un paese immerso nel verde e tutte le...
1,0,165,0_perchè molto_perchè_perchè sporchi_perche,"[perchè molto, perchè, perchè sporchi, perche,...",[perché si viveva discretamente il sindaco ci ...
2,1,158,1_ambiente risparmio_sostenibilità risparmio e...,"[ambiente risparmio, sostenibilità risparmio e...",[sostenibilità risparmio attenzione all'ambien...
3,2,123,2_molti parchi_parchi molto_parchi parchi_parchi,"[molti parchi, parchi molto, parchi parchi, pa...","[molti parchi, non ci sono molti parchi, non c..."
4,3,109,3_pochi spazi verdi_spazi verdi pochi_pochi sp...,"[pochi spazi verdi, spazi verdi pochi, pochi s...","[pochi spazi, ci sono pochi spazi verdi, ci so..."
5,4,92,4_saprei nient_saprei saprei_saprei saprei so_...,"[saprei nient, saprei saprei, saprei saprei so...","[non saprei, non saprei, non so non saprei]"
6,5,89,5_pulito_pulita_puliti_pulito poca,"[pulito, pulita, puliti, pulito poca, abbastan...","[e pulito, e pulito, pulizia]"
7,6,79,6_uh_uiguho_uayg_oi,"[uh, uiguho, uayg, oi, oo, uiguho hhfhoghgo, o...","[uh, uh, mm guydga uyg uygef uayg fuaeg ufygas..."
8,7,74,7_molto verde_verde molto_tanto verde molto_ab...,"[molto verde, verde molto, tanto verde molto, ...","[c’è molto verde, molto verde, verde]"
9,8,68,8_potrebbe migliorare_migliorare potrebbe_potr...,"[potrebbe migliorare, migliorare potrebbe, pot...","[potrebbe migliorare, si potrebbe migliorare, ..."


In [9]:
topic_model.visualize_barchart()

In [10]:
data_path = "/content/datasets/json/3.json"
sentences = opt.build_data(data_path)
topic_model.visualize_documents(sentences)
# If you do not see the plot properly please click on "Reset Axes" icon at the top right corner of the plot

In [11]:
topic_model.visualize_topics()

In [12]:
topic_model.visualize_heatmap()

In [13]:
sentence_topic_df

,text,topic
0,non saperei,4
1,attenzione all'ambiete,-1
2,telegreenriscaldamento,1
3,sostenibilità risparmio attenzione all'ambiente,1
4,basso costo,1
...,...,...
1551,c’è tutto,-1
1552,perché si vive bene e ce molta cura,0
1553,andrebbero descritte davvero moltissime cose...,8
1554,perché potrebbe migliorare,-1


### Example 4

In [11]:
get_ipython().system_raw("mlflow ui --port 5000 &")

run_name = "bert-topic-forth-dataset-1"
label_lower = 25
label_upper = 35
max_evals = 20
penalty= 0.3
experiment_name = "Forth-Dataset-Model-BERT-1"
data_path = "/content/datasets/json/4.json"
tracking_uri = "http://127.0.0.1:5000"

topic_model, sentence_topic_df = opt.clustering_eval_mlflow(
                                                tracking_client = tracking_uri,
                                                run_name = run_name,
                                                sentences = None,
                                                data_path = data_path,
                                                label_lower = label_lower,
                                                label_upper = label_upper,
                                                penalty = penalty,
                                                max_evals = max_evals,
                                                experiment_name = experiment_name)


Experiment name set on MLFlow Tracking Server: 
 Forth-Dataset-Model-BERT-1
100%|██████████| 20/20 [27:29<00:00, 82.49s/trial, best loss: 0.3261464750171116]
<Run: data=<RunData: metrics={'label_count': 5.0, 'loss': 0.3261464750171116}, params={'min_cluster_size': '300',
 'n_components': '28',
 'n_neighbors': '68',
 'random_state': '42'}, tags={'mlflow.runName': 'bert-topic-forth-dataset-1',
 'mlflow.source.name': '/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'root'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/382878145999407882/5e7fc0ee2d4d485aa5857073e5e88c4a/artifacts', end_time=1699947329058, experiment_id='382878145999407882', lifecycle_stage='active', run_id='5e7fc0ee2d4d485aa5857073e5e88c4a', run_name='bert-topic-forth-dataset-1', run_uuid='5e7fc0ee2d4d485aa5857073e5e88c4a', start_time=1699947329010, status='FINISHED', user_id='root'>, inputs=<RunInputs: dataset_inputs=[]>>


Instructions for updating:
Use tf.identity with explicit device placement instead.


In [ ]:
topic_model.get_topic_info()

In [13]:
topic_model.visualize_barchart()

In [14]:
data_path = "/content/datasets/json/4.json"
sentences = opt.build_data(data_path)
topic_model.visualize_documents(sentences)

#### Unpack biggest cluster and apply again the same procedure to uncover hidden topics

In [ ]:
list(sentence_topic_df[sentence_topic_df['topic']==0]['text'])

In [26]:
run_name = "bert-topic-forth-dataset-2"
label_lower = 25
label_upper = 35
max_evals = 20
penalty= 0.3
experiment_name = "Forth-Dataset-Model-BERT-1"
data_path = "/content/datasets/json/4.json"
tracking_uri = "http://127.0.0.1:5000"

topic_model, sentence_topic_df = opt.clustering_eval_mlflow(
                                                tracking_client = tracking_uri,
                                                run_name = run_name,
                                                sentences = list(sentence_topic_df[sentence_topic_df['topic']==0]['text']),
                                                data_path = None,
                                                label_lower = label_lower,
                                                label_upper = label_upper,
                                                penalty = penalty,
                                                max_evals = max_evals,
                                                experiment_name = experiment_name)

Experiment name set on MLFlow Tracking Server: 
 Forth-Dataset-Model-BERT-1
100%|██████████| 20/20 [15:28<00:00, 46.44s/trial, best loss: 0.39993439755084187]
<Run: data=<RunData: metrics={'label_count': 13.0, 'loss': 0.39993439755084187}, params={'min_cluster_size': '40',
 'n_components': '82',
 'n_neighbors': '41',
 'random_state': '42'}, tags={'mlflow.runName': 'bert-topic-forth-dataset-2',
 'mlflow.source.name': '/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'root'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/382878145999407882/677a886dcad34e9db057f40d1b906d2c/artifacts', end_time=1699949711772, experiment_id='382878145999407882', lifecycle_stage='active', run_id='677a886dcad34e9db057f40d1b906d2c', run_name='bert-topic-forth-dataset-2', run_uuid='677a886dcad34e9db057f40d1b906d2c', start_time=1699949711740, status='FINISHED', user_id='root'>, inputs=<RunInputs: dataset_inputs=[]>>


In [27]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,420,-1_edizioni cashback_cashback rimborso maggior...,"[edizioni cashback, cashback rimborso maggiore...","[più edizioni di cashback, più edizioni cashba..."
1,0,2542,0_ancora ricevuto rimborso_rimborso ancora ric...,"[ancora ricevuto rimborso, rimborso ancora ric...","[non ho ancora ricevuto il rimborso, non ho an..."
2,1,392,1_prezzi pannolini_pannolini pampers_pannolini...,"[prezzi pannolini, pannolini pampers, pannolin...","[cashback su tutti i prodotti pampers, il prez..."
3,2,265,2_cashback po alto_cashback alto_alto cashback...,"[cashback po alto, cashback alto, alto cashbac...","[cashback più alto, un cashback più alto, un c..."
4,3,214,3_volte possibilità partecipare_possibilità pa...,"[volte possibilità partecipare, possibilità pa...","[possibilità di partecipare più volte, la poss..."
5,4,131,4_sconti maggiori_maggiori sconti_maggiore sco...,"[sconti maggiori, maggiori sconti, maggiore sc...","[più sconto, più sconto, più sconto]"
6,5,123,5_maggiore rimborso_rimborso maggiore_maggiore...,"[maggiore rimborso, rimborso maggiore, maggior...","[maggiore rimborso, maggiore rimborso, maggior..."
7,6,110,6_soglie basse_basse soglie_basse soglie basse...,"[soglie basse, basse soglie, basse soglie bass...","[soglie più basse, soglie più basse, soglie pi..."
8,7,108,7_meno punti_punti extra_punti premi punti_sog...,"[meno punti, punti extra, punti premi punti, s...","[più punti, più punti, più punti]"
9,8,97,8_piu prodotti_altri prodotti_coinvolti piu pr...,"[piu prodotti, altri prodotti, coinvolti piu p...","[più prodotti, più prodotti, più prodotti]"


In [28]:
topic_model.visualize_barchart()